In [7]:
#코랩으로 실행 
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from dotenv import load_dotenv
import os
import csv

load_dotenv()

client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")

print("SPOTIPY_CLIENT_ID:", client_id)
print("SPOTIPY_CLIENT_SECRET:", client_secret)
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

SPOTIPY_CLIENT_ID: 1fe3ef9478bf4379b37e9cda4de800ba
SPOTIPY_CLIENT_SECRET: d5d98eef06eb47b8af42a717b1639e92


In [26]:
# 아티스트 id로 top5트랙 검색 예제 
lz_uri = 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'

results = sp.artist_top_tracks(lz_uri)

# get top 5 tracks
for track in results['tracks'][:5]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
    print()

track    : Left and Right (Feat. Jung Kook of BTS)
audio    : https://p.scdn.co/mp3-preview/9e4342eefa4f33df4e9b199e60dbce0f59e46eff?cid=1fe3ef9478bf4379b37e9cda4de800ba
cover art: https://i.scdn.co/image/ab67616d0000b27335d2e0ed94a934f2cc46fa49

track    : My Universe
audio    : https://p.scdn.co/mp3-preview/0790eb03d0d2b6aac7c9c762bc2bf1f3649dd7ca?cid=1fe3ef9478bf4379b37e9cda4de800ba
cover art: https://i.scdn.co/image/ab67616d0000b273ec10f247b100da1ce0d80b6d

track    : Take Two
audio    : https://p.scdn.co/mp3-preview/ef43da30a06fcad16898e9fff1cadd0ccf953fc9?cid=1fe3ef9478bf4379b37e9cda4de800ba
cover art: https://i.scdn.co/image/ab67616d0000b2738a701e76e8845928f6cd81c8

track    : Dynamite
audio    : https://p.scdn.co/mp3-preview/98e266fea9df84fa3e5ca84934c513211e89489b?cid=1fe3ef9478bf4379b37e9cda4de800ba
cover art: https://i.scdn.co/image/ab67616d0000b273c07d5d2fdc02ae252fcd07e5

track    : Dreamers [Music from the FIFA World Cup Qatar 2022 Official Soundtrack]
audio    : https://

In [45]:
#Spotify는 한국가수 필터링이 안되므로 k-pop장르를 하나라도 가진 unique 가수 크롤링
track_ids = []

for year in [2022, 2023]:
        for i in range(0, 1000, 50):
            track_results = sp.search(q='year:{year} genre:k-pop', type='track', limit=50, offset=i)
            for t in track_results['tracks']['items']:
                # Extract relevant information
                track_id = t['id']
                title = t['name']
                artist_name = t['artists'][0]['name']
                artist_id = t['artists'][0]['id']

                print(f"Track ID: {track_id}")
                print(f"Title: {title}")
                print(f"Artist: {artist_name}")
                print(f"Artist id: {artist_id}")
                print()

                track_ids.append(track_id)

# Save data to a CSV file
csv_file_path = "kpop_tracks.csv"
fields = ["Track ID", "Title", "Artist Name", "Artist ID"]

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    
    # Write the header
    writer.writerow(fields)
    
    # Write the unique artist data
    writer.writerows(unique_artists)

print(f"Unique artist data has been saved to {csv_file_path}")

Unique artist data has been saved to kpop_tracks_2022.csv


In [49]:
# 가수만 크롤링 
unique_artists = set()

for i in range(0, 1000, 50):
    track_results = sp.search(q='year:2022 genre:k-pop', type='track', limit=50, offset=i)
    for t in track_results['tracks']['items']:
        artist_id = t['artists'][0]['id']
        artist_name = t['artists'][0]['name']

        unique_artists.add((artist_id, artist_name))
        
for i in range(0, 1000, 50):
    track_results = sp.search(q='year:2023 genre:k-pop', type='track', limit=50, offset=i)
    for t in track_results['tracks']['items']:
        artist_id = t['artists'][0]['id']
        artist_name = t['artists'][0]['name']

        unique_artists.add((artist_id, artist_name))

csv_file_path = "unique_artists.csv"
fields = ["Artist ID", "Artist Name"]

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    
    # Write the header
    writer.writerow(fields)
    
    # Write the unique artist data
    writer.writerows(unique_artists)

print(f"Unique artist data has been saved to {csv_file_path}")

Unique artist data has been saved to unique_artists.csv


In [51]:
# Load unique artist data from CSV file
csv_file_path = "unique_artists.csv"
unique_artists = set()

with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        artist_id = row["Artist ID"]
        artist_name = row["Artist Name"]
        unique_artists.add((artist_id, artist_name))

# List to store track information
track_data = []

# Search for tracks released by unique artists in 2022 and 2023
for year in [2022, 2023]:
    for artist_id, artist_name in unique_artists:
        track_results = sp.artist_top_tracks(artist_id, country='US')
        for track in track_results['tracks']:
            # Check if the track is from the specified year
            if track['album']['release_date'][:4] == str(year):
                # Extract relevant information
                track_id = track['id']
                title = track['name']
                release_date = track['album']['release_date']
                
                audio_features = sp.audio_features([track_id])[0]
                bpm = audio_features['tempo'] if audio_features and 'tempo' in audio_features else None

                # Append information to the track_data list
                track_data.append({
                    "Release Date": release_date,
                    "Title": title,
                    "Artist Name": artist_name,
                    "BPM": bpm
                })

# Save track data to a CSV file
csv_file_path = "track_data.csv"
fields = ["Release Date", "Title", "Artist Name", "BPM"]

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fields)

    # Write the header
    writer.writeheader()

    # Write the track data
    writer.writerows(track_data)

print(f"Track data has been saved to {csv_file_path}")

Track data has been saved to track_data.csv
